<a href="https://colab.research.google.com/github/nicholasketo/Stock-Market-Value-Prediction/blob/main/Quant_momentum_investing_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install xlsxwriter

In [2]:
import numpy as np
import pandas as pd
import requests as rq
import math
from scipy import stats
import xlsxwriter


In [3]:
url = 'https://raw.githubusercontent.com/nicholasketo/Stock-Market-Value-Prediction/main/sp_500_stocks.csv'
stocks = pd.read_csv(url)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
IEX_CLOUD_API_TOKEN = 'sk_30a7d5719bf14aa584faa9c117f2988f'

In [5]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
try:
    response = rq.get(api_url)
    response.raise_for_status()  # Check for any HTTP errors
    data = response.json()  # Try to parse the response as JSON

    # If you want to print the data
    print(data)

except rq.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except ValueError as e:
    print(f"ValueError: {e}")

{'companyName': 'Apple Inc', 'marketcap': 2827801368640, 'week52high': 197.7, 'week52low': 123.48, 'week52highSplitAdjustOnly': 198.23, 'week52lowSplitAdjustOnly': 124.17, 'week52change': 0.31235149250930383, 'sharesOutstanding': 15552752000, 'float': 0, 'avg10Volume': 63492450, 'avg30Volume': 58016722, 'day200MovingAvg': 179.3, 'day50MovingAvg': 174.76, 'employees': 164000, 'ttmEPS': 6.13, 'ttmDividendRate': 0.7084164673375632, 'dividendYield': 0.00389625160784052, 'nextDividendDate': '2023-11-10', 'exDividendDate': '2023-08-11', 'nextEarningsDate': '2024-02-01', 'peRatio': 29.154094217640083, 'beta': 0, 'maxChangePercent': 69.87946358958365, 'year5ChangePercent': 2.626391160397304, 'year2ChangePercent': 0.2171838347918296, 'year1ChangePercent': 0.3178313115038076, 'ytdChangePercent': 0.4072363308762295, 'month6ChangePercent': 0.05181499116931243, 'month3ChangePercent': 0.01935324721923215, 'month1ChangePercent': 0.02439574060510452, 'day30ChangePercent': 0.02439574060510452, 'day5Cha

In [6]:
data['year1ChangePercent']

0.3178313115038076

In [14]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [21]:

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = rq.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol in data and 'quote' in data[symbol] and 'stats' in data[symbol]:
            final_dataframe = final_dataframe.append(
                pd.Series([symbol,
                           data[symbol]['quote'].get('latestPrice', 'N/A'),
                           data[symbol]['stats'].get('year1ChangePercent', 'N/A'),
                           'N/A'],
                          index=my_columns),
                ignore_index=True
            )
        else:
            print(f"Data not found for symbol: {symbol}")

final_dataframe

<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

Data not found for symbol: ABC
Data not found for symbol: ANTM


<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

Data not found for symbol: DISCA
Data not found for symbol: FBHS


<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

...

Data not found for symbol: NLOK
Data not found for symbol: RE


<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

Data not found for symbol: VIAC
Data not found for symbol: WLTW


<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-21-a55b969dd3c4>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

,Symbol,Price,Year Change Percent,N/A
0,A,109.530,-0.193502,N/A
1,AAL,12.170,-0.179469,N/A
2,AAP,58.010,-0.670976,N/A
3,AAPL,182.160,0.317831,N/A
4,ABBV,142.985,-0.001331,N/A
...,...,...,...,...
491,YUM,125.690,0.043362,N/A
492,ZBH,104.420,-0.007333,N/A
493,ZBRA,207.195,-0.119817,N/A
494,ZION,33.680,-0.285414,N/A


In [26]:
final_dataframe.sort_values('Year Change Percent', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Symbol,Price,Year Change Percent,N/A
0,270,LB,79.920,2.281287,N/A
1,339,NVDA,464.020,2.215377,N/A
2,364,PHM,81.620,1.141211,N/A
3,8,ADBE,584.730,0.953662,N/A
4,48,AVGO,903.700,0.949811,N/A
5,30,AMD,113.320,0.79851,N/A
6,384,RCL,95.570,0.787349,N/A
7,408,SNPS,503.710,0.778847,N/A
8,190,FTI,20.745,0.741398,N/A
9,83,CDNS,254.750,0.737877,N/A


In [31]:
def portfolio_input():
  global portfolio_size
  portfolio_size = input('Enter the size of your portfolio: ')
  try:
    float(portfolio_size)
  except ValueError:
    print('That is not a number! \nPlease try again: ')
    portfolio_size = input('Enter the size of your portfolio: ')


portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 1000000
1000000


In [38]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/ final_dataframe.loc[i, 'Price'])

final_dataframe

,index,Symbol,Price,Year Change Percent,N/A,Number of Shares to Buy
0,270,LB,79.920,2.281287,N/A,250.0
1,339,NVDA,464.020,2.215377,N/A,43.0
2,364,PHM,81.620,1.141211,N/A,245.0
3,8,ADBE,584.730,0.953662,N/A,34.0
4,48,AVGO,903.700,0.949811,N/A,22.0
5,30,AMD,113.320,0.79851,N/A,176.0
6,384,RCL,95.570,0.787349,N/A,209.0
7,408,SNPS,503.710,0.778847,N/A,39.0
8,190,FTI,20.745,0.741398,N/A,964.0
9,83,CDNS,254.750,0.737877,N/A,78.0


In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy'
    'One-Year Price Return'
]